In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargar los datos
raw_data = pd.read_csv('../datasets/loan_default_data.csv', encoding='latin1')
raw_data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,2,0.0,0.0,0.0,36 months,0.00,0.00,< 1 year,...,1.0,0.000000,0.00,0.00,0.00,Jan-07,0.00,Jan-07,Jan-07,1
1,3,545583,703644,2500.0,2500.0,2500.0,36 months,13.98,85.42,4 years,...,10.0,3075.291779,3075.29,2500.00,575.29,Jul-13,90.85,Aug-13,Jun-16,0
2,4,532101,687836,5000.0,5000.0,5000.0,36 months,15.95,175.67,4 years,...,15.0,2948.760000,2948.76,1909.02,873.81,Nov-11,175.67,NaN,Mar-12,1
3,5,877788,1092507,7000.0,7000.0,7000.0,36 months,9.91,225.58,10+ years,...,20.0,8082.391880,8082.39,7000.00,1082.39,Mar-14,1550.27,NaN,Mar-14,0
4,6,875406,1089981,2000.0,2000.0,2000.0,36 months,5.42,60.32,10+ years,...,15.0,2161.663244,2161.66,2000.00,161.66,Feb-14,53.12,NaN,Jun-16,0


In [3]:
# Eliminar columnas innecesarias
columns_to_drop = ['Unnamed: 0', 'zip_code', 'member_id', 'id', 'last_credit_pull_d',
                   'next_pymnt_d', 'earliest_cr_line', 'inq_last_6mths', 'revol_bal',
                   'revol_util', 'total_acc', 'repay_fail', 'total_acc', 'earliest_cr_line',
                   'addr_state', 'funded_amnt', 'funded_amnt_inv', 'total_pymnt_inv']
raw_data = raw_data.drop(columns=columns_to_drop)
raw_data.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,...,dti,delinq_2yrs,mths_since_last_delinq,open_acc,pub_rec,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt
0,0.0,36 months,0.00,0.00,< 1 year,RENT,0.0,Not Verified,Dec-99,Charged Off,...,0.00,0.0,NaN,0.0,0.0,0.000000,0.00,0.00,Jan-07,0.00
1,2500.0,36 months,13.98,85.42,4 years,RENT,20004.0,Not Verified,Jul-10,Does not meet the credit policy. Status:Fully ...,...,19.86,0.0,NaN,7.0,0.0,3075.291779,2500.00,575.29,Jul-13,90.85
2,5000.0,36 months,15.95,175.67,4 years,RENT,59000.0,Not Verified,Jun-10,Charged Off,...,19.57,0.0,59.0,7.0,0.0,2948.760000,1909.02,873.81,Nov-11,175.67
3,7000.0,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,Sep-11,Fully Paid,...,10.80,3.0,3.0,7.0,0.0,8082.391880,7000.00,1082.39,Mar-14,1550.27
4,2000.0,36 months,5.42,60.32,10+ years,RENT,30000.0,Not Verified,Sep-11,Fully Paid,...,3.60,0.0,72.0,7.0,0.0,2161.663244,2000.00,161.66,Feb-14,53.12


In [4]:
# Eliminamos los registros que nos daban problemas
raw_data = raw_data.drop([0, 54], axis=0)

In [5]:
# Reemplazamos los NaNs por 0 en emp_length, asumiendo desempleo
raw_data['emp_length'] = raw_data['emp_length'].fillna(0)

In [6]:
# Reemplazamos los NaN en mths_since_last_delinq por 200, ya que el maximo de la columna es 120
raw_data['mths_since_last_delinq'] = raw_data['mths_since_last_delinq'].fillna(200)

# Reemplazamos los NaN en last_pymnt_d por la fecha de issue_d
raw_data['last_pymnt_d'] = raw_data['last_pymnt_d'].fillna(raw_data['issue_d'])

In [7]:
# Crear columna 'long_term' que indica si el préstamo es a largo plazo o no
raw_data['long_term'] = raw_data['term'].apply(lambda x: 1 if '60' in x else 0)
raw_data = raw_data.drop(columns=['term'])

In [8]:
# Eliminar outliers en income: cualquier valor superior a 1.5 millones
raw_data = raw_data[raw_data['annual_inc'] < 1.5e6]

In [9]:
# Crear columna boolean 'employment_verified'
raw_data['employment_verified'] = raw_data['verification_status'].apply(lambda x: 1 if x == 'Verified' else 0)
# Eliminar la columna 'verification_status'
raw_data = raw_data.drop(columns=['verification_status'])

In [10]:
# Consolidar la columna 'loan_status', los valores son:
# Fully Paid, Charged Off, Does not meet the credit policy. Status:Fully Paid,
# Current, Does not meet the credit policy. Status:Charged Off, Late (31-120 days)
# In Grace Period, Late (16-30 days), y Default
# Fully Paid, Does not meet the credit policy. Status:Fully Paid deben ser Paid
# Charged Off, Does not meet the credit policy. Status:Charged Off y Default deben ser Charged Off
# Late (31-120 days), Late (16-30 days) deben ser Late
# Current y In Grace Period deben ser Current
raw_data['loan_status'] = raw_data['loan_status'].replace(
    ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'], 'Paid')
raw_data['loan_status'] = raw_data['loan_status'].replace(
    ['Charged Off', 'Does not meet the credit policy. Status:Charged Off', 'Default'], 'Charged Off')
raw_data['loan_status'] = raw_data['loan_status'].replace(
    ['Late (31-120 days)', 'Late (16-30 days)'], 'Late')
raw_data['loan_status'] = raw_data['loan_status'].replace(
    ['Current', 'In Grace Period'], 'Current')

In [11]:

# Crear time_to_delinq, si el Prestamo esta pagado o sin novedades, este valor
# sera NaN, de otra manera, sera la diferencia entre la fecha de emision y
# la fecha del ultimo pago
raw_data['issue_d'] = pd.to_datetime(raw_data['issue_d'], format='%b-%y')
raw_data['last_pymnt_d'] = pd.to_datetime(raw_data['last_pymnt_d'], format='%b-%y')
mask_not_paid_off = raw_data['loan_status'].isin(['Charged Off', 'Late'])
raw_data.loc[mask_not_paid_off, 'time_to_delinq'] = (
    raw_data.loc[mask_not_paid_off, 'last_pymnt_d'] - raw_data.loc[mask_not_paid_off, 'issue_d']).dt.days
raw_data.loc[~mask_not_paid_off, 'time_to_delinq'] = np.nan
raw_data['time_to_delinq'] = raw_data['time_to_delinq'].astype('Int64')
# EDIT Apr-05: Mantenemos la columna last_pymnt_d temporalmente
#raw_data = raw_data.drop(columns=['last_pymnt_d'])

In [12]:
raw_data.head()

,loan_amnt,int_rate,installment,emp_length,home_ownership,annual_inc,issue_d,loan_status,purpose,dti,...,open_acc,pub_rec,total_pymnt,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,long_term,employment_verified,time_to_delinq
1,2500.0,13.98,85.42,4 years,RENT,20004.0,2010-07-01,Paid,other,19.86,...,7.0,0.0,3075.291779,2500.00,575.29,2013-07-01,90.85,0,0,<NA>
2,5000.0,15.95,175.67,4 years,RENT,59000.0,2010-06-01,Charged Off,debt_consolidation,19.57,...,7.0,0.0,2948.760000,1909.02,873.81,2011-11-01,175.67,0,0,518
3,7000.0,9.91,225.58,10+ years,MORTGAGE,53796.0,2011-09-01,Paid,other,10.80,...,7.0,0.0,8082.391880,7000.00,1082.39,2014-03-01,1550.27,0,0,<NA>
4,2000.0,5.42,60.32,10+ years,RENT,30000.0,2011-09-01,Paid,debt_consolidation,3.60,...,7.0,0.0,2161.663244,2000.00,161.66,2014-02-01,53.12,0,0,<NA>
5,3600.0,10.25,116.59,10+ years,MORTGAGE,675048.0,2010-04-01,Paid,other,1.55,...,8.0,0.0,4206.031191,3600.00,606.03,2013-05-01,146.75,0,0,<NA>


In [13]:
# Guardar el dataset limpio
raw_data.to_csv('../datasets/cleaned_loan_data.csv', index=False)